# Research review

Problem: Papers to be presented at a conference are reviewed by experts who decide if the paper will be accepted. We want to automate this process s.t.:

- papers are reviewed by at least two experts
- experts can bid on papers:
  - *yes* = the expert would like to review the paper
  - *maybe* = the expert has no strong preference for this paper
  - *no* = the expert does not want to review the paper
- the workload (assigned papers) of each expert should be approximately be equal, i.e. not differ more than 1
- if an expert bids *no* then this paper will not be assigned to this expert
- the number of *yes* votes from experts assigned to a paper should be maximized, i.e. if possible we are respecting the bid of the experts

In [1]:
%%file ./instances/review_input.lp

expert(a;b;c;d;e).
paper(p1;p2;p3;p4;p5;p6;p7;p8;p9).
bid(
    a,p1,yes; a,p2,yes; a,p3,maybe; a,p4,maybe; a,p5,yes; a,p6,no; a,p7,no; a,p8,no; a,p9,yes;
    b,p1,yes; b,p2,yes; b,p3,no; b,p4,no; b,p5,maybe; b,p6,no; b,p7,yes; b,p8,maybe; b,p9,no;
    c,p1,no; c,p2,no; c,p3,yes; c,p4,no; c,p5,yes; c,p6,no; c,p7,no; c,p8,no; c,p9,yes;
    d,p1,no; d,p2,no; d,p3,yes; d,p4,yes; d,p5,yes; d,p6,maybe; d,p7,maybe; d,p8,no; d,p9,maybe;
    e,p1,yes; e,p2,no; e,p3,maybe; e,p4,no; e,p5,maybe; e,p6,yes; e,p7,no; e,p8,yes; e,p9,no;
).

Overwriting ./instances/review_input.lp


Solver:

In [2]:
%%file ./instances/review.lp

#const min_reviewer = 2.

% assign papers
{assigned(P,E): expert(E)} >= min_reviewer :- paper(P).

% no assignment if expert said no
:- assigned(P,E), bid(E,P,no).

% workload balanced
work(E,N) :- N = #count{P: assigned(P,E)}, expert(E).
:- work(E1,N1), work(E2,N2), E1 != E2, |N1-N2|>1.

% maximize yes
#maximize{1,P,E: assigned(P,E), bid(E,P,yes)}.

#show assigned/2.


Overwriting ./instances/review.lp


In [3]:
!clingo ./instances/review_input.lp ./instances/review.lp 0

clingo version 5.6.2
Reading from ./instances/review_input.lp ...
Solving...
Answer: 1
assigned(p2,a) assigned(p3,a) assigned(p4,a) assigned(p9,a) assigned(p1,b) assigned(p2,b) assigned(p7,b) assigned(p8,b) assigned(p3,c) assigned(p5,c) assigned(p9,c) assigned(p4,d) assigned(p6,d) assigned(p7,d) assigned(p9,d) assigned(p1,e) assigned(p5,e) assigned(p6,e) assigned(p8,e)
Optimization: -12
Answer: 2
assigned(p2,a) assigned(p3,a) assigned(p4,a) assigned(p9,a) assigned(p1,b) assigned(p2,b) assigned(p7,b) assigned(p8,b) assigned(p3,c) assigned(p5,c) assigned(p9,c) assigned(p3,d) assigned(p4,d) assigned(p6,d) assigned(p7,d) assigned(p1,e) assigned(p5,e) assigned(p6,e) assigned(p8,e)
Optimization: -13
Answer: 3
assigned(p1,a) assigned(p2,a) assigned(p4,a) assigned(p9,a) assigned(p1,b) assigned(p2,b) assigned(p7,b) assigned(p8,b) assigned(p3,c) assigned(p5,c) assigned(p9,c) assigned(p3,d) assigned(p4,d) assigned(p6,d) assigned(p7,d) assigned(p1,e) assigned(p5,e) assigned(p6,e) assigned(p8,e)
Op

The optimal assignement results in 14 *yes*:

- p1: a, b , e
- p2: a, b
- p3: c, d
- p4: a, d
- p5: c, e
- p6: d, e
- p7: b, d
- p8: b, e
- p9: a, c